# 資料預處理

In [1]:
import os
os.chdir("C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\Code")
import sys
sys.path.append("C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\Code")

from Get_file import Get_file   # return dirpath, dirnames, filenames
from FFT import FFT, PlotDataOnTimeDomain
from BandPassFilter import BandPassFilter, PlotDataOnFreqDomain, BandPassListFilter
import pyedflib
import numpy as np
import datetime

import matplotlib.pyplot as plt
from sklearn.decomposition import FastICA, PCA

%matplotlib inline

data_path = "C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\database"  # EEG Data direction
BandPassICAFigureSavingDirection = "C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\Code\\Fig"
SaveFFTAllDataDirection = "C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\Code\\Fig\\FFTFig_AllData"

# Setting parameters
SampleFrequency = 128
DEBUG = True
LabelSize = 40
LowerFreqCut = 1
HigherFreqCut = 50
BandPassOrder = 5
channel_we_use = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]   # 0~39 共40, 14 channels we use
SAVE_FIG = False


### 讀取檔案

In [2]:
#Four Lables
normal_label = []   # 0
scare_label = []    # 1
touch_label = []    # 2
laugh_label = []    # 3

#Four Signal
normal_signal = []
scare_signal = []
touch_signal = []
laugh_signal = []

signal = []
labels = []

_,data_files,_ = Get_file(data_path)
if data_files == []:
    raise("Not found any files ! check your data_path")
else:
    print('Get data from database ...')
    for data in data_files:
        _,_,file_names = Get_file(data_path+'\\'+data)
        for fn in file_names:
            if fn.split('.')[1] == 'edf':
                fedf = pyedflib.EdfReader(data_path+'\\'+data+'\\'+fn)
                n = fedf.signals_in_file
                signal_labels = np.asarray(fedf.getSignalLabels())
                sigbufs = np.zeros((n, fedf.getNSamples()[0]))
                for i in np.arange(n):
                    sigbufs[i, :] = fedf.readSignal(i)

                if 'normal' in fn:
                    normal_label.append(0)
                    normal_signal.append(sigbufs)
                elif 'scare' in fn:
                    scare_label.append(1)
                    scare_signal.append(sigbufs)
                elif 'touch' in fn:
                    touch_label.append(2)
                    touch_signal.append(sigbufs)
                elif 'laugh' in fn:
                    laugh_label.append(3)
                    laugh_signal.append(sigbufs)
                    
                if DEBUG:
                    print(sigbufs.shape)

''' Labels --> 40 channels
['COUNTER' 'INTERPOLATED' 'AF3' 'F7' 'F3' 'FC5' 'T7' 'P7' 'O1' 'O2' 'P8'
 'T8' 'FC6' 'F4' 'F8' 'AF4' 'RAW_CQ' 'GYROX' 'GYROY' 'MARKER'
 'MARKER_HARDWARE' 'SYNC' 'TIME_STAMP_s' 'TIME_STAMP_ms' 'CQ_AF3' 'CQ_F7'
 'CQ_F3' 'CQ_FC5' 'CQ_T7' 'CQ_P7' 'CQ_O1' 'CQ_O2' 'CQ_P8' 'CQ_T8' 'CQ_FC6'
 'CQ_F4' 'CQ_F8' 'CQ_AF4' 'CQ_CMS' 'CQ_DRL']
'''

print('Okay')
if DEBUG:
    print('normal signal length : ', len(normal_signal))
    print('scare signal length : ', len(scare_signal))
    print('touch signal length : ', len(touch_signal))
    print('laugh signal length : ', len(laugh_signal))


Get data from database ...
(40, 38400)
(40, 105344)
(40, 38400)
(40, 105344)
(40, 179328)
(40, 309760)
(40, 38400)
(40, 105728)
(40, 158976)
(40, 58112)
(40, 50944)
(40, 33152)
(40, 3968)
(40, 38400)
(40, 105344)
(40, 178944)
(40, 308992)
(40, 38400)
(40, 105728)
(40, 179840)
(40, 237312)
(40, 38400)
(40, 105600)
(40, 179584)
(40, 38400)
(40, 178944)
(40, 308864)
(40, 38400)
(40, 105344)
(40, 179072)
(40, 83328)
(40, 117632)
(40, 107392)
(40, 38400)
(40, 308992)
(40, 38400)
(40, 179072)
(40, 105344)
(40, 308736)
(40, 38400)
(40, 313472)
(40, 38400)
(40, 105344)
(40, 38400)
(40, 105344)
(40, 38528)
(40, 179200)
(40, 308736)
(40, 38400)
(40, 179328)
(40, 38400)
(40, 105344)
Okay
normal signal length :  16
scare signal length :  11
touch signal length :  9
laugh signal length :  16


In [3]:
def GrabEmotion(Signal, TimeSecondsi, TimeSecondsf):
    pass

In [4]:
def CheckListTheSameSize(ListData):
    di = 0
    dsum = 0
    for i in range(len(ListData)):
        if i == 0:
            pass
        else:
            dsum += (ListData[i].shape == ListData[i-1].shape)
            
    if np.sum(dsum) != (len(ListData)-1):
        return False
    else:
        return True
    
def CalculateZeros(NumpyArray):
    LogicWithZeros = (NumpyArray == 0)
    return np.sum(LogicWithZeros)

def PlotFreqencyFigure(FreqList, FFTData, FreqL, FreqH, YlimL=None, YlimH=None, title=None, FigSize=(12,8)):
    plt.figure(figsize=FigSize)
    plt.plot(FreqList, FFTData, '*-')
    plt.xlabel('Freq(Hz)')
    plt.ylabel('micro volt (mv)')
    plt.xlim(FreqL, FreqH)
    if YlimL is None and YlimH is None:
        pass
    else:
        plt.ylim(YlimL, YlimH)
    if title is not None:
        plt.title(title)
        
    else:
        plt.title('FreqencyFigure')
    plt.grid(True)
    plt.show()

### Analyze steps : Filters ==> FFT ==> Build Model

In [130]:
# Filter, Each Data 128*5 second to as a period
# Return Data Shape : [(Channels, TimestepsData, Features)]  --> List, each list data is data numbers
def BandPassSPlitAllData(SignalListData, LowerFreqCut, HigherFreq, fs, order=5, PlotFlag=False):
    channel_we_use = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]   # 0~39 共40, 14 channels we use
    SamplesPerSecond = fs
    DataLength = SignalListData[0].shape[1]
    SignalFreqX = 0
    ReturnSignalList = []
    for i in range(len(SignalListData)):
        LabelNum = SignalListData[i].shape[0]
        FeaturesNum = SignalListData[i].shape[1]
        TimesToSplit = int(FeaturesNum/SamplesPerSecond)
        print('Features Numbers : ', FeaturesNum)
        print('Times to Split Numbers :', TimesToSplit)
        CHCNT = 0
        for ch in range(LabelNum):
            if ch in channel_we_use:
                SignalData = np.zeros([14, 5, 64], dtype=np.float32)
                Times = 0
                FiveTimesToSplitBuff = np.zeros([5, 64], dtype=np.float32)   # Five Seconds Data Buffer
                FiveSecondCNT = 0
                TimestepCNT = 0
                for t in range(TimesToSplit):
                    ChannelData = BandPassFilter(SignalListData[i][ch][Times:Times+SamplesPerSecond],
                                                         LowerFreqCut, HigherFreqCut, fs, order=order)
                    SignalFreqX, FFTChannelData = FFT(ChannelData, SampleFrequency, data_length=128)
                    Times += SamplesPerSecond
                    FiveTimesToSplitBuff[FiveSecondCNT, :] = FFTChannelData
                    FiveSecondCNT += 1
                    if FiveSecondCNT > 4:
                        SignalData[CHCNT, :, :] = FiveTimesToSplitBuff
                        FiveTimesToSplitBuff = np.zeros([5, 64], dtype=np.float32)   # Five Seconds Data Buffer
                        FiveSecondCNT = 0
                        
                    if PlotFlag:
                        if t % 10 == 0:
                            PlotFreqencyFigure(SignalFreqX, FFTChannelData, 8, 30)
                if CalculateZeros(SignalData) != 0:
                    print("Warning: Have Zero Values in the Numpy array !!!")
                ReturnSignalList.append(SignalData)
                TimestepCNT = 0
                CHCNT += 1


    return SignalFreqX, ReturnSignalList

In [ ]:
# Filter, Each Data 128*5 second to as a period
# Return Data Shape : [(Channels, TimestepsData, Features)]  --> List, each list data is data numbers
def BandPassOneChanalData(SignalListData, LowerFreqCut, HigherFreq, channel, fs, order=5, PlotFlag=False):
    channel_we_use = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]   # 0~39 共40, 14 channels we use
    SamplesPerSecond = fs
    DataLength = SignalListData[0].shape[1]
    SignalFreqX = 0
    ReturnSignalList = []
    for i in range(len(SignalListData)):
        LabelNum = SignalListData[i].shape[0]
        FeaturesNum = SignalListData[i].shape[1]
        TimesToSplit = int(FeaturesNum/SamplesPerSecond)
        print('Features Numbers : ', FeaturesNum)
        print('Times to Split Numbers :', TimesToSplit)
        CHCNT = 0
        for ch in range(LabelNum):
            if ch == channel_we_use[channel]:
                SignalData = np.zeros([14, 5, 64], dtype=np.float32)
                Times = 0
                FiveTimesToSplitBuff = np.zeros([5, 64], dtype=np.float32)   # Five Seconds Data Buffer
                FiveSecondCNT = 0
                TimestepCNT = 0
                for t in range(TimesToSplit):
                    ChannelData = BandPassFilter(SignalListData[i][ch][Times:Times+SamplesPerSecond],
                                                         LowerFreqCut, HigherFreqCut, fs, order=order)
                    SignalFreqX, FFTChannelData = FFT(ChannelData, SampleFrequency, data_length=128)
                    Times += SamplesPerSecond
                    FiveTimesToSplitBuff[FiveSecondCNT, :] = FFTChannelData
                    FiveSecondCNT += 1
                    if FiveSecondCNT > 4:
                        SignalData[CHCNT, :, :] = FiveTimesToSplitBuff
                        FiveTimesToSplitBuff = np.zeros([5, 64], dtype=np.float32)   # Five Seconds Data Buffer
                        FiveSecondCNT = 0

                    if PlotFlag:
                        if t % 10 == 0:
                            PlotFreqencyFigure(SignalFreqX, FFTChannelData, 8, 30)
                if CalculateZeros(SignalData) != 0:
                    print("Warning: Have Zero Values in the Numpy array !!!")
                ReturnSignalList.append(SignalData)
                TimestepCNT = 0
                CHCNT += 1


    return SignalFreqX, ReturnSignalList

In [131]:
# Return data is a list type
# Each list data is a array type which shape is (Channels, TimesSplit, Features)
# Warning TimeSpilt maybe not the same, so I use list to save the data
SignalFreqX, NormalSignalList = BandPassSPlitAllData(normal_signal, LowerFreqCut, HigherFreqCut, SampleFrequency, order=BandPassOrder, PlotFlag=False)
SignalFreqX, ScareSignalList = BandPassSPlitAllData(scare_signal, LowerFreqCut, HigherFreqCut, SampleFrequency, order=BandPassOrder, PlotFlag=False)
SignalFreqX, TouchSignalList = BandPassSPlitAllData(touch_signal, LowerFreqCut, HigherFreqCut, SampleFrequency, order=BandPassOrder, PlotFlag=False)
SignalFreqX, LaughSignalList = BandPassSPlitAllData(laugh_signal, LowerFreqCut, HigherFreqCut, SampleFrequency, order=BandPassOrder, PlotFlag=False)

if DEBUG:
    # Signal Data Numbers
    print('Normal Data length:', len(NormalSignalList))
    print('Scare Data lentgh:', len(ScareSignalList))
    print('Touch Data length:', len(TouchSignalList))
    print('Laugh Data length:', len(LaughSignalList))
    

Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300


KeyboardInterrupt: 

In [129]:
ScareSignalList.shape

(14, 5, 64)

In [8]:
# Hyper parameter Setup
TEST_SIZE = 0.3
BATCH_SIZE = 128        # betch training per 128 as a batch
TIME_STEP = 5*64   # per five seconds
INPUT_SIZE = 64    # 64 Features
LEARNING_RATE = 0.01

raw_X = NormalSignalList[5]
raw_y = NormalSignalList[5]

In [9]:
# Normalize and to Split the training data, testing data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# def Split_data(raw_X, raw_y, test_size, Standard=True):
#     train_X, test_X, train_y, test_y = train_test_split(raw_X, raw_y, test_size=test_size, shuffle=True)
#     if Standard:
#         sc = StandardScaler()
#         sc.fit(train_X)
#         train_X = sc.transform(train_X)
#         test_X = sc.transform(test_X)
#     self.X = train_X
#     self.y = train_y
#     return train_X, test_X, train_y, test_y

train_X, test_X, train_y, test_y = train_test_split(raw_X, raw_y, test_size=TEST_SIZE)
std = StandardScaler()


# Build LSTM RNN Model

In [10]:
# import LSTM Package using keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.recurrent import SimpleRNN, LSTM

C:\Users\zxpay\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
# RNN/LSTM: input format (batch_size, timesteps, input_dim)
model = Sequential()
model.add(LSTM(units=20, input_shape=(TIME_STEP, INPUT_SIZE)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

In [12]:
# See the NN Configure
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                6800      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 6,821
Trainable params: 6,821
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Training Data now
model.fit(SignalNormalTrain, LabelNormal, epochs=100, batch_size=BATCH_SIZE, verbose=2)

Epoch 1/100
 - 2s - loss: 0.0946
Epoch 2/100
 - 0s - loss: 0.0911
Epoch 3/100
 - 0s - loss: 0.0873
Epoch 4/100
 - 0s - loss: 0.0833
Epoch 5/100
 - 0s - loss: 0.0793
Epoch 6/100
 - 0s - loss: 0.0753
Epoch 7/100
 - 0s - loss: 0.0714
Epoch 8/100
 - 0s - loss: 0.0675
Epoch 9/100
 - 0s - loss: 0.0638
Epoch 10/100
 - 0s - loss: 0.0601
Epoch 11/100
 - 1s - loss: 0.0566
Epoch 12/100
 - 0s - loss: 0.0532
Epoch 13/100
 - 0s - loss: 0.0499
Epoch 14/100
 - 0s - loss: 0.0467
Epoch 15/100
 - 0s - loss: 0.0437
Epoch 16/100
 - 0s - loss: 0.0407
Epoch 17/100
 - 0s - loss: 0.0379
Epoch 18/100
 - 0s - loss: 0.0353
Epoch 19/100
 - 0s - loss: 0.0327
Epoch 20/100
 - 0s - loss: 0.0303
Epoch 21/100
 - 0s - loss: 0.0280
Epoch 22/100
 - 0s - loss: 0.0258
Epoch 23/100
 - 0s - loss: 0.0237
Epoch 24/100
 - 0s - loss: 0.0217
Epoch 25/100
 - 0s - loss: 0.0199
Epoch 26/100
 - 0s - loss: 0.0182
Epoch 27/100
 - 0s - loss: 0.0165
Epoch 28/100
 - 0s - loss: 0.0150
Epoch 29/100
 - 0s - loss: 0.0136
Epoch 30/100
 - 0s - lo